In [363]:
# Install a pip package in the current Jupyter kernel
# import sys
# !{sys.executable} -m pip install BeautifulSoup4
# python -m pip install BeautifulSoup4

In [364]:

# pip install BeautifulSoup4

In [365]:
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# from bs4 import BeautifulSoup
#  http://beautiful-soup-4.readthedocs.io/en/latest/    # for more advanced web scraping  

import lxml
import html5lib
import csv

import time  # used to record time to run functions

from collections import Counter

print('Libraries imported.')

Libraries imported.


## Import neighborhoods data

start with Toronto

In [366]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_to_neighborhoods = pd.read_html(url, attrs={"class": "wikitable"})[0]   # 0 is for the 1st table in this particular page
df_to_neighborhoods.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


clean the Toronto dataframe

In [367]:
df_to_neighborhoods['Borough']=df_to_neighborhoods['Borough'].replace('Not assigned',np.NaN)   # replace 'Not assigned' with NaN 
df_to_neighborhoods=df_to_neighborhoods.dropna() 
df_to_neighborhoods.reset_index(drop=True, inplace=True)
print("Dropped 'Not assigned' values in the 'Postal Code' column!")

# pd.set_option('display.max_rows', None)     # show all data in dataframe

# check for mispelled 'Not assigned' values in 'Borough' which has not been replaced
print("Any mispelles not assigned values in 'Borough'?  " + str(df_to_neighborhoods['Borough'][df_to_neighborhoods['Borough'].str.lower().str.contains('t as', regex=False)]) + ", " + str(df_to_neighborhoods['Borough'][df_to_neighborhoods['Borough'].str.lower().str.contains('not ', regex=False)]) )   

# find non unique (duplicate) postal codes 
dupli= "no"
for d in df_to_neighborhoods['Postal Code'].duplicated().unique():
    if d == True:
        dupli = ""
print("There is " + dupli+" duplicate 'Postal Code' rows!")


# find neighborhoods with 'Not assigned' values
assigned="Cannot find any"
for a in df_to_neighborhoods['Neighbourhood'].isin(['Not assigned']):
    if a == True:
        assigned="Found"
print(assigned+" 'Not assigned' values in the 'Neighbourhood' column!")

df_to_neighborhoods.head()

Dropped 'Not assigned' values in the 'Postal Code' column!
Any mispelles not assigned values in 'Borough'?  Series([], Name: Borough, dtype: object), Series([], Name: Borough, dtype: object)
There is no duplicate 'Postal Code' rows!
Cannot find any 'Not assigned' values in the 'Neighbourhood' column!


,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [368]:
df_to_neighborhoods.shape

(103, 3)

### Add location data of the neighborhoods in Toronto

Google Maps Geocoding API started to cost therefore geocoder is used, however it is a bit unreliable it seems, thus we need to download an .CSV file with data of the coordinates

In [369]:
df_to_coordinates=pd.read_csv('../datafiles/Geospatial_Coordinates.csv')
df_to_coordinates

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [370]:
import geocoder # import geocoder
import geopy
from geopy.geocoders import Nominatim

In [385]:

# Coordinate arrays
lat_list=np.array([])
long_list=np.array([])

for i, pcode in enumerate(df_to_neighborhoods['Postal Code']):
    # print(i, pcode)
    # initialize your variable to None
    lat_coords =None
    lng_coords =None
    

    locator = Nominatim(user_agent="myGeocoder")
    count = 0
    # loop until you get the coordinates
    while(lng_coords is None):
        location = locator.geocode('{}, Toronto, Ontario'.format(pcode))
        try:
            lat_coords = location.latitude
            lng_coords = location.longitude
            # print("Coordinates: "+ str(lat_coords) + ", "+str(lng_coords) )
        except:
            lat_coords =None
            lng_coords =None

        if count == 3:  # the limit
            # print(" Error 'Location' not found in Geocode! Switching to CSV file")
            lat_coords = df_to_coordinates.loc[ df_to_coordinates['Postal Code'] == pcode ]['Latitude'].to_list()[0]
            lng_coords = df_to_coordinates.loc[ df_to_coordinates['Postal Code'] == pcode ]['Longitude'].to_list()[0]
            # print("\nCoordinates: "+ str(lat_coords) + ", "+str(lng_coords) )
        # if count == 3:
        #     print("Error count = 3")
        #     break

        count = count + 1
    
    lat_to_list= np.append(lat_list, lat_coords)
    long_to_list= np.append(long_list, lng_coords)

# lat_to_list

In [386]:

lat_to_list.shape

(1,)

In [387]:
df_to_neighborhoods["Latitude"]=lat_to_list
df_to_neighborhoods["Longitude"]=long_to_list
df_to_neighborhoods

ValueError: Length of values (1) does not match length of index (103)

### Now adding tempreature average of the city New York
We assume that the intercity difference in temperature is not too high.

first we read data from wikipedia 

In [372]:
url = 'https://en.wikipedia.org/wiki/List_of_cities_by_average_temperature'
df_city_weather = pd.read_html(url, attrs={"class": "wikitable"})[0]   # 0 is for the 1st table in this particular page
df_city_weather.head()

,Country,City,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year,Ref.
0,Algeria,Algiers,11.2(52.2),11.9(53.4),12.8(55.0),14.7(58.5),17.7(63.9),21.3(70.3),24.6(76.3),25.2(77.4),23.2(73.8),19.4(66.9),15.2(59.4),12.1(53.8),17.4(63.3),[1]
1,Algeria,Tamanrasset,12.8(55.0),15.0(59.0),18.1(64.6),22.2(72.0),26.1(79.0),28.9(84.0),28.7(83.7),28.2(82.8),26.5(79.7),22.4(72.3),17.3(63.1),13.9(57.0),21.7(71.1),[2]
2,Algeria,Reggane,16.0(60.8),18.2(64.8),23.1(73.6),27.9(82.2),32.2(90.0),36.4(97.5),39.8(103.6),38.4(101.1),35.5(95.9),29.2(84.6),22.0(71.6),17.8(64.0),28.3(82.9),[3]
3,Angola,Luanda,26.7(80.1),28.5(83.3),28.6(83.5),28.2(82.8),27.0(80.6),23.9(75.0),22.1(71.8),22.1(71.8),23.5(74.3),25.2(77.4),26.7(80.1),26.9(80.4),25.8(78.4),[4]
4,Benin,Cotonou,27.3(81.1),28.5(83.3),28.9(84.0),28.6(83.5),27.8(82.0),26.5(79.7),25.8(78.4),25.6(78.1),26.0(78.8),26.7(80.1),27.6(81.7),27.3(81.1),27.2(81.0),[5]


Clean the data

In [373]:
# remove the farenheit temperature values
weather_months = df_city_weather.columns.drop(['Country','City','Ref.']) # list of months

for month in  weather_months:
    df_city_weather[str(month)] = df_city_weather[str(month)].str.replace(r"\(.*\)","") # Remove the parenthesis + the within

# drop the ref column
df_city_weather = df_city_weather.drop(['Ref.'], axis=1) 

# the cities average temperatures in celsius
df_city_weather.head()

,Country,City,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
0,Algeria,Algiers,11.2,11.9,12.8,14.7,17.7,21.3,24.6,25.2,23.2,19.4,15.2,12.1,17.4
1,Algeria,Tamanrasset,12.8,15.0,18.1,22.2,26.1,28.9,28.7,28.2,26.5,22.4,17.3,13.9,21.7
2,Algeria,Reggane,16.0,18.2,23.1,27.9,32.2,36.4,39.8,38.4,35.5,29.2,22.0,17.8,28.3
3,Angola,Luanda,26.7,28.5,28.6,28.2,27.0,23.9,22.1,22.1,23.5,25.2,26.7,26.9,25.8
4,Benin,Cotonou,27.3,28.5,28.9,28.6,27.8,26.5,25.8,25.6,26.0,26.7,27.6,27.3,27.2


In [374]:
url = 'https://www.numbeo.com/crime/rankings.jsp'
df_city_weather = pd.read_html(url)[1]   # 0 is for the 1st table in this particular page
# df_city_weather.head()
df_city_weather.drop(['Rank'], axis=1, inplace=True) # drop rank dol since only nan
df_city_weather['Rank'] = df_city_weather.index # add index as rank, since they are ordered by rank
# df_city_weather['City'].str.split(',', expand=True)[2].unique()
df_city_weather=pd.concat([df_city_weather, df_city_weather['City'].str.split(', ', expand=True)], axis=1)
# df_city_weather
# ~pd.notna(df_city_weather[2])

In [375]:
# fixa 
# print(Counter(~pd.notna(df_city_weather[2])))
# print(Counter(df_city_weather[2]))
# print(Counter(df_city_weather[1]))
# df_city_weather[pd.notna(df_city_weather[2])][1] =  df_city_weather[pd.notna(df_city_weather[2])][1].astype(str) + ",  "
# df_city_weather[~pd.notna(df_city_weather[2])][2].str.replace(None,"a")
df_city_weather.loc[pd.notna(df_city_weather[2]), 2]=", "+df_city_weather.loc[pd.notna(df_city_weather[2]), 2].astype(str)
df_city_weather[2].fillna("",inplace=True)
# df_city_weather.head(20)
# # df_city_weather[2]

df_city_weather['Country']=df_city_weather[1].str.cat(df_city_weather[2])
# # # # df_city_weather[df_city_weather[1]!=""]
# # # # df_city_weather[df_city_weather[1]==None]
# # # df_city_weather[pd.notna(df_city_weather[2])][1] 

df_city_weather.drop([1,2,"City"],axis=1, inplace=True)
df_city_weather.rename(columns={0:'City'},inplace=True)
print(df_city_weather.head(20))


    Crime Index  Safety Index  Rank              City              Country
0         84.57         15.43     0           Caracas            Venezuela
1         81.89         18.11     1          Pretoria         South Africa
2         81.35         18.65     2    San Pedro Sula             Honduras
3         81.05         18.95     3      Port Moresby     Papua New Guinea
4         80.93         19.07     4  Pietermaritzburg         South Africa
5         80.80         19.20     5            Durban         South Africa
6         80.31         19.69     6      Johannesburg         South Africa
7         78.51         21.49     7         Fortaleza               Brazil
8         77.42         22.58     8    Rio de Janeiro               Brazil
9         77.22         22.78     9             Kabul          Afghanistan
10        76.85         23.15    10    Port Elizabeth         South Africa
11        76.81         23.19    11            Recife               Brazil
12        75.79         2

In [376]:
# print(df_city_weather[[0,1,2, 'Country']].head(20))
# print(Counter(df_city_weather['Country']))
# print(df_city_weather.head(20))

In [377]:
A=['aa','adsadaf', 'fafwfe','de', 'Ecuador']
eb=pd.DataFrame({'col':A})
eb['col2'] = 'd ' + eb['col'].astype(str)
eb

,col,col2
0,aa,d aa
1,adsadaf,d adsadaf
2,fafwfe,d fafwfe
3,de,d de
4,Ecuador,d Ecuador


In [378]:
df_SE_region_mobility_report=pd.read_csv('../datafiles/2020_SE_Region_Mobility_Report.csv')
# print(df_coordinates['date'].isna().sum(), df_coordinates['date'].count())
df_SE_region_mobility_report

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,SE,Sweden,NaN,NaN,NaN,NaN,NaN,2020-02-15,2.0,0.0,3.0,4.0,-2.0,0.0
1,SE,Sweden,NaN,NaN,NaN,NaN,NaN,2020-02-16,-10.0,-7.0,-30.0,-11.0,-5.0,2.0
2,SE,Sweden,NaN,NaN,NaN,NaN,NaN,2020-02-17,-4.0,-3.0,-3.0,-6.0,-6.0,2.0
3,SE,Sweden,NaN,NaN,NaN,NaN,NaN,2020-02-18,0.0,0.0,13.0,-4.0,-5.0,1.0
4,SE,Sweden,NaN,NaN,NaN,NaN,NaN,2020-02-19,3.0,1.0,26.0,-3.0,-5.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53730,SE,Sweden,Västra Götaland County,Vårgårda Municipality,NaN,NaN,NaN,2020-10-12,NaN,NaN,NaN,-29.0,-11.0,NaN
53731,SE,Sweden,Västra Götaland County,Vårgårda Municipality,NaN,NaN,NaN,2020-10-13,NaN,NaN,NaN,-20.0,-15.0,NaN
53732,SE,Sweden,Västra Götaland County,Vårgårda Municipality,NaN,NaN,NaN,2020-10-14,NaN,NaN,NaN,-27.0,-10.0,NaN
53733,SE,Sweden,Västra Götaland County,Vårgårda Municipality,NaN,NaN,NaN,2020-10-15,NaN,NaN,NaN,-20.0,-10.0,NaN


In [379]:

# df_coordinates=pd.read_csv('../datafiles/airbnb_listings_sum_stockholm.csv')
# # print(df_coordinates['date'].isna().sum(), df_coordinates['date'].count())
# df_coordinates

In [380]:
# count=0
# airbnb_calend_list=[]
# with open('../datafiles/airbnb_calendar_stockholm.csv', newline='',encoding = 'cp850') as csvfile:
#     csv_reader_data = csv.reader(csvfile)
#     for row in csv_reader_data:
#         if np.shape(row)[0]==1:
#             redone_csv_reading = csv.reader(row)
#             for new_row in redone_csv_reading:
#                 next_row =[new_row]
#         elif np.shape(row)[0]==7:
#              next_row = [new_row]
#         airbnb_calend_list.append(next_row)         

In [381]:
start_time = time.time()
wrong_amount_of_element=[]
count=0
airbnb_calend_list=[]

with open('../datafiles/airbnb_calendar_stockholm.csv', newline='',encoding = 'cp850') as csvfile:
    csv_reader_data = csv.reader(csvfile)
    for row in csv_reader_data: 
        next_row = []
        if count >=1 and np.shape(row)[0]==1: # if count is not header and has "" around the all elements
            # print(row)
            # print(np.shape(row))

            redone_csv_reading = csv.reader(row)
            for new_row in redone_csv_reading:
                # print([new_row])
                # print(np.shape(new_row))
                if np.shape(new_row)[0]==7:
                    next_row = new_row
                    # print("1")
                    # print(new_row)
                else:
                    wrong_amount_of_element.append(np.shape(new_row)[0])
                    # print("\nError: not shape of 'new_row' ==7 ")
                    # print(new_row)

            # print("\n")
        elif count >=1 and np.shape(row)[0]==7:
            next_row = row
            # print("2")
            # print([next_row])
        elif count==0:
            # print(row)
            # print(np.shape(row))
            headers_row = row

        # else:
            # print("\nError: not shape of 'row' == 1 or 7")
            # print(row)
            # next_row = np.empty((1,7,))
            # next_row[:] = np.nan
            # # next_row=np.nan
            # print("")


        if count >=1:     
            airbnb_calend_list.append(next_row)

        count = count + 1
        if count == 15000000:
            break
# print(airbnb_calend_list)
print(np.shape(airbnb_calend_list),np.shape(headers_row))
df_airbnb_calend_stockholm = pd.DataFrame(data=airbnb_calend_list, columns= headers_row)
print(df_airbnb_calend_stockholm)
# df_airbnb_calend_stockholm.to_csv('../datafiles/airbnb_calendar_stockholm_new.csv',index=False)
print("Number of elements in row and there count: ",Counter(wrong_amount_of_element))
print("--- %s seconds ---" % (time.time() - start_time))

(1048575, 7) (7,)
        listing_id        date available      price adjusted_price  \
0           565912  2020-06-27         t  $2,461.00      $2,461.00   
1           145320  2020-06-27         f  $1,424.00      $1,424.00   
2           145320  2020-06-28         f  $1,424.00      $1,424.00   
3           145320  2020-06-29         f  $1,424.00      $1,424.00   
4           145320  2020-06-30         f  $1,424.00      $1,424.00   
...            ...         ...       ...        ...            ...   
1048570   15773429  2020-11-24         t    $250.00        $250.00   
1048571   15773429  2020-11-25         t    $250.00        $250.00   
1048572   15773429  2020-11-26         t    $250.00        $250.00   
1048573   15773429  2020-11-27         t    $250.00        $250.00   
1048574   15773429  2020-11-28         t    $250.00        $250.00   

        minimum_nights maximum_nights  
0                    4             20  
1                    3           1125  
2                    

In [ ]:
start_time = time.time()
wrong_amount_of_element=[]
number_of_rows=16

count=0
airbnb_calend_list=[]
with open('../datafiles/airbnb_listings_sum_stockholm.csv', newline='',encoding = 'cp850') as csvfile:
    csv_reader_data = csv.reader(csvfile)
    for row in csv_reader_data: 
        next_row = []

        if count >=1 and np.shape(row)[0]==1: # if count is not header and has "" around the all elements
            # print(row)
            # print(np.shape(row))
            row[0]=row[0].replace("\n","")
            redone_csv_reading = csv.reader(row)
            for new_row in redone_csv_reading:
                # print([new_row])
                # print(np.shape(new_row))
                if np.shape(new_row)[0]==number_of_rows:
                    next_row = new_row
                    # print("1")
                    # print(new_row)
                else:
                    wrong_amount_of_element.append(np.shape(new_row)[0])
                    # print("\nError: not shape of 'new_row' ==7 ")
                    # print(new_row)
                    # print(np.shape(new_row)[0])

            # print("\n")
        elif count >=1 and np.shape(row)[0]==number_of_rows:
            next_row = row
            # print("2")
            # print([next_row])
        elif count==0:
            # print(row)
            # print(np.shape(row))
            headers_row = row

        # else:
            # print("\nError: not shape of 'row' == 1 or 7")
            # print(row)
            # next_row = np.empty((1,7,))
            # next_row[:] = np.nan
            # # next_row=np.nan
            # print("")


        if count >=1:     
            airbnb_calend_list.append(next_row)

        count = count + 1
        if count == 10000:
            break
# print(airbnb_calend_list)
# print(np.shape(airbnb_calend_list),np.shape(headers_row))
df_airbnb_listing_stockholm = pd.DataFrame(data=airbnb_calend_list, columns= headers_row)
print(df_airbnb_listing_stockholm)
# df_airbnb_calend_stockholm.to_csv('../datafiles/airbnb_calendar_stockholm_new.csv',index=False)
print("Number of elements in row and there count: ",Counter(wrong_amount_of_element))
print("--- %s seconds ---" % (time.time() - start_time))

In [13]:
df_airbnb_listing_stockholm['neighbourhood'].unique()

array(['S├Âdermalms', None, 'Norrmalms', 'Kungsholmens', 'Rinkeby-Tensta',
       'Farsta', '├älvsj├Â', '├ûstermalms', 'Skarpn├ñcks',
       'Enskede-├àrsta-Vant├Ârs', 'H├ñgersten-Liljeholmens', 'Bromma',
       'Sk├ñrholmens', 'H├ñsselby-V├ñllingby', 'Sp├Ñnga-Tensta'],
      dtype=object)

In [14]:
# # Correctly spell the names of the neighborhoods
# df_airbnb_listing_stockholm['neighbourhood'].replace('S├Âdermalms','Södermalm', inplace=True)
# df_airbnb_listing_stockholm['neighbourhood'].replace('Norrmalms','Norrmalm', inplace=True)
# df_airbnb_listing_stockholm['neighbourhood'].replace('Kungsholmens','Kungsholmen', inplace=True)
# df_airbnb_listing_stockholm['neighbourhood'].replace('├älvsj├Â','Älvsjö', inplace=True)
# df_airbnb_listing_stockholm['neighbourhood'].replace('├ûstermalms','Östermalm', inplace=True)
# df_airbnb_listing_stockholm['neighbourhood'].replace('H├ñgersten-Liljeholmens','Hägersten-Liljeholmen', inplace=True)
# df_airbnb_listing_stockholm['neighbourhood'].replace('Sk├ñrholmens','Skärholmen', inplace=True)
# df_airbnb_listing_stockholm['neighbourhood'].replace('H├ñsselby-V├ñllingby','Hässelby-Vällingby', inplace=True)
# df_airbnb_listing_stockholm['neighbourhood'].replace('Sp├Ñnga-Tensta','Spånga-Tensta', inplace=True)

In [15]:
# df_test=df_airbnb_calend_stockholm[df_airbnb_calend_stockholm['neighbourhood']=='Södermalm']
# df_test

In [16]:
# df_test[df_test['name'].str.lower().str.contains("fantastic sofo")]   

It seems like the prices are in SEK - after comparing 'price' to website price for same place

In [17]:
start_time = time.time()
wrong_amount_of_element=[]
number_of_rows=6

count=0
airbnb_calend_list=[]
with open('../datafiles/airbnb_reviews_stockholm.csv', newline='',encoding = 'cp850') as csvfile:
    csv_reader_data = csv.reader(csvfile)
    for row in csv_reader_data: 
        next_row = [] 
        # if count== 177:
        #     print(row)
        #     print(row[0])
        #     print(np.shape(row))
            # redone_csv_reading = csv.reader(row)
            # for new_row in redone_csv_reading:
            #     # print([new_row])
            #     # print(np.shape(new_row))
            #     next_row = new_row

            #     print(next_row)
        if count >=1 and np.shape(row)[0]==1: # if count is not header and has "" around the all elements
            # print(row)
            # print(np.shape(row))
            # print(row[0])
            # print(np.shape(row[0]))
            row[0]=row[0].replace("\n","")
            redone_csv_reading = csv.reader(row)
            for new_row in redone_csv_reading:
                # print(new_row)
                # print(np.shape(new_row))
                if np.shape(new_row)[0]==number_of_rows:
                    next_row = new_row
                    # print("1")
                    # print(new_row)
                else:
                    wrong_amount_of_element.append(np.shape(new_row)[0])
                    # print("\nError: not shape of 'new_row' ==7 ")
                    # print(new_row)

            # print("\n")
        elif count >=1 and np.shape(row)[0]==number_of_rows:
            next_row = row
            # print("2")
            # print([next_row])
        elif count==0:
            # print(row)
            # print(np.shape(row))
            headers_row = row

        # else:
            # print("\nError: not shape of 'row' == 1 or 7")
            # print(row)
            # next_row = np.empty((1,7,))
            # next_row[:] = np.nan
            # # next_row=np.nan
            # print("")


        if count >=1:     
            airbnb_calend_list.append(next_row)


        count = count + 1
        if count == 1000000:
            break
# print(airbnb_calend_list)
print(np.shape(airbnb_calend_list),np.shape(headers_row))
df_airbnb_review_stockholm = pd.DataFrame(data=airbnb_calend_list, columns= headers_row)
print(df_airbnb_review_stockholm)
# df_airbnb_calend_stockholm.to_csv('../datafiles/airbnb_calendar_stockholm_new.csv',index=False)
print("Number of elements in row and there count: ",Counter(wrong_amount_of_element))
print("--- %s seconds ---" % (time.time() - start_time))

(182014,) (6,)
       listing_id         id        date reviewer_id reviewer_name  \
0          145320  156423122  2017-05-30   123386382         Erwin   
1          145320  170211906  2017-07-15   123091743          Anne   
2          145320  172169175  2017-07-20       78004      Patricia   
3          145320  176647581  2017-07-31   103178743     Charlotte   
4          145320  185676021  2017-08-22     4023961     Alexander   
...           ...        ...         ...         ...           ...   
182009   43720290  629441507  2020-06-14   308451513            Lu   
182010   43720290  631087136  2020-06-21    76954984        Shahin   
182011   43724610  630964346  2020-06-21    38355251          Ines   
182012   43749955  632003080  2020-06-25    92774441       Babette   
182013   43840216  630977479  2020-06-21   251908973          Dala   

                                      comments;;;;;;;;;;;  
0       Prima plek om Stockholm te bekijken. Alles is ...  
1       Cosy and clean f